In [75]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [76]:
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


premLeague = "https://sports.williamhill.com/betting/en-gb/football/competitions/OB_TY295/English-Premier-League/matches/OB_MGMB/Match-Betting"

driver = webdriver.Firefox()
driver.get(premLeague)
matches = driver.find_elements(By.CSS_SELECTOR, ".sp-o-market__title > a")
matches_links = []

# Get links for the matches
# [[match, link]]
for match in matches:
    matches_links.append([match.text, match.get_attribute('href')])

driver.close()

# For each link, we fetch the player data
# create an object [name, odds, match]
# details = [   name, match,
#              {"first":[first[0],first[1]]} ,
# {"anytime":[anytime[0],anytime[1]]},
# {"last":[last[0],last[1]]},
# {"two_or_more":[two_or_more[0],two_or_more[1]]},
# {"hattrick":[hattrick[0],hattrick[1]]},
# {"first_and_last":[first_and_last[0],first_and_last[1]]},
# {"first_or_last":[first_or_last[0],first_or_last[1]]}
# ]
details = []
for link in matches_links:  # Here you're getting data for each link .i.e each loop has one match
    driver = webdriver.Firefox()
    driver.get(link[1])
    players = driver.find_elements(
        By.CSS_SELECTOR, "article.cpt-goalscorer__player")

    if (len(players) == 0):  # check if the details are loaded. Otherwise, it is in the following GW
        continue

    players_details = []

    for index, tag in enumerate(players):  # for each match get player details
        # Get player name
        player = tag.find_element(
            By.TAG_NAME, 'h3').get_attribute("textContent")
        odds = tag.find_elements(By.CLASS_NAME, 'selectiondetails')

        def splitOdds(odd):
            if (len(odd.split('/')) > 1):
                return odd.split('/')
            else:
                return [odd]

        # Get players odds
        if (index+1 == len(players)):
            # For the no goal scored odd
            first = odds[0].get_attribute("data-odds")
            odss_details = {"name": player, "match": link[0], "first": first}

            players_details.append(odss_details)
            continue
        else:
            first = odds[0].get_attribute("data-odds")
            anytime = odds[1].get_attribute("data-odds")
            last = odds[2].get_attribute("data-odds")
            two_or_more = odds[3].get_attribute("data-odds")
            hattrick = odds[4].get_attribute("data-odds")
            first_and_last = odds[5].get_attribute("data-odds")
            first_or_last = odds[6].get_attribute("data-odds")

            odss_details = {
                "name": player,
                "match": link[0],
                "first": first,
                "anytime": anytime,
                "last": last,
                "two_or_more": two_or_more,
                "hattrick": hattrick,
                "first_and_last": first_and_last,
                "first_or_last": first_or_last
            }
            players_details.append(odss_details)

    # group the match details
    details.append(players_details)

    driver.close()


print(details)
print(matches_links)

[[{'name': 'Ollie Watkins', 'match': 'Aston Villa v Fulham', 'first': '4/1', 'anytime': '19/20', 'last': '10/3', 'two_or_more': '5/1', 'hattrick': '18/1', 'first_and_last': '10/1', 'first_or_last': '7/4'}, {'name': 'Moussa Diaby', 'match': 'Aston Villa v Fulham', 'first': '11/2', 'anytime': '8/5', 'last': '9/2', 'two_or_more': '9/1', 'hattrick': '40/1', 'first_and_last': '14/1', 'first_or_last': '13/5'}, {'name': 'Jhon Duran', 'match': 'Aston Villa v Fulham', 'first': '11/2', 'anytime': '2/1', 'last': '11/2', 'two_or_more': '12/1', 'hattrick': '50/1', 'first_and_last': '16/1', 'first_or_last': '16/5'}, {'name': 'Bertrand Traore', 'match': 'Aston Villa v Fulham', 'first': '13/2', 'anytime': '12/5', 'last': '13/2', 'two_or_more': '14/1', 'hattrick': '70/1', 'first_and_last': '20/1', 'first_or_last': '19/5'}, {'name': 'Leon Bailey', 'match': 'Aston Villa v Fulham', 'first': '7/1', 'anytime': '11/5', 'last': '6/1', 'two_or_more': '14/1', 'hattrick': '60/1', 'first_and_last': '18/1', 'first

In [77]:
import pandas as pd
len(det)
init_data = {"name": [],
             "match": [],
             "first": [],
             "anytime": [],
             "last": [],
             "two_or_more": [],
             "hattrick": [],
             "first_and_last": [],
             "first_or_last": []}

df = pd.DataFrame(init_data)

for items in details:  # for each match in object
    for item in items:  # for each player in match
        df.loc[len(df)] = item

df.to_excel("../data/scrapped/williamhill.xlsx", sheet_name="GW12")